In [9]:
import pandas as pd
import numpy as np

DATA_DIR = "./data/raw" 

accounts = pd.read_csv(f"{DATA_DIR}/accounts.csv")
contacts = pd.read_csv(f"{DATA_DIR}/contacts.csv")
leads = pd.read_csv(f"{DATA_DIR}/leads.csv")
opps = pd.read_csv(f"{DATA_DIR}/opportunities.csv")
acts = pd.read_csv(f"{DATA_DIR}/activities.csv")

def _snakecase_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns.str.strip()
                 .str.lower()
                 .str.replace(" ", "_")
                 .str.replace("-", "_")
    )
    return df

accounts = _snakecase_cols(accounts)
contacts = _snakecase_cols(contacts)
leads = _snakecase_cols(leads)
opps = _snakecase_cols(opps)
acts = _snakecase_cols(acts)

print("Rows:", {
    "accounts": len(accounts),
    "contacts": len(contacts),
    "leads": len(leads),
    "opportunities": len(opps),
    "activities": len(acts)
})

print("\nOpportunities — column check (top 25):")
print(list(opps.columns)[:25])

# --- Fast missingness snapshot for fields that will drive EV & leakage ---
key_fields = [
    "opportunity_id", "amount", "win_probability_estimated", "expected_close_date",
    "stage", "stage_entered_date", "last_stage_change_date", "created_date",
    "days_in_stage", "days_since_last_activity"
]
present = [c for c in key_fields if c in opps.columns]

missing_snapshot = (opps[present].isna().mean().sort_values(ascending=False) * 100).round(2)
print("\nOpportunities — % missing (relevant fields):")
print(missing_snapshot)

# --- Basic distribution sanity for amount and win prob (if present) ---
if "amount" in opps.columns:
    print("\nAmount sanity:")
    print(opps["amount"].describe(percentiles=[.1, .25, .5, .75, .9, .95]).round(2))

if "win_probability_estimated" in opps.columns:
    print("\nWin probability sanity:")
    print(opps["win_probability_estimated"].describe(percentiles=[.1, .25, .5, .75, .9, .95]).round(4))

opp_key = None
for k in ["opportunity_id", "opp_id", "oppty_id"]:
    if k in opps.columns:
        opp_key = k
        break

act_opp_key = None
for k in ["opportunity_id", "opp_id", "oppty_id"]:
    if k in acts.columns:
        act_opp_key = k
        break

if opp_key and act_opp_key:
    act_counts = acts.groupby(act_opp_key).size().rename("activity_count").reset_index()
    opps_prof = opps[[opp_key]].merge(act_counts, how="left", left_on=opp_key, right_on=act_opp_key)
    opps_prof["activity_count"] = opps_prof["activity_count"].fillna(0).astype(int)

    print("\nActivity coverage check (opportunity-level):")
    print(opps_prof["activity_count"].describe().round(2))
    print("Pct with 0 activities:", round((opps_prof["activity_count"] == 0).mean() * 100, 2), "%")
else:
    print("\nNOTE: Could not find a common opportunity key between opportunities and activities.")
    print("Opportunities keys tried: opportunity_id / opp_id / oppty_id")
    print("Activities keys tried: opportunity_id / opp_id / oppty_id")

Rows: {'accounts': 500, 'contacts': 1416, 'leads': 1500, 'opportunities': 900, 'activities': 3879}

Opportunities — column check (top 25):
['opp_id', 'lead_id', 'account_id', 'created_date', 'stage', 'amount', 'expected_close_date', 'outcome', 'win_probability_estimated', 'stage_jump_flag', 'backward_stage_change_count', 'days_in_stage_current']

Opportunities — % missing (relevant fields):
expected_close_date          30.00
amount                       12.56
win_probability_estimated     0.00
stage                         0.00
created_date                  0.00
dtype: float64

Amount sanity:
count       787.00
mean      26400.22
std       16919.51
min        3515.00
10%       10399.20
25%       14825.50
50%       22598.00
75%       32536.00
90%       46903.60
95%       57051.70
max      131704.00
Name: amount, dtype: float64

Win probability sanity:
count    900.0000
mean       0.4547
std        0.1913
min        0.0500
10%        0.1910
25%        0.3202
50%        0.4561
75%        

In [10]:
import pandas as pd
import numpy as np

# 1) Standardize core fields
opps_m2 = opps.copy()

# Safety: ensure numeric
opps_m2["win_probability_estimated"] = pd.to_numeric(opps_m2["win_probability_estimated"], errors="coerce")
opps_m2["amount"] = pd.to_numeric(opps_m2["amount"], errors="coerce")

# 2) Stage-median imputation for missing amount (explainable, not ML)
stage_medians = opps_m2.groupby("stage")["amount"].median()
overall_median = opps_m2["amount"].median()

opps_m2["amount_imputed_flag"] = opps_m2["amount"].isna()

opps_m2["amount_imputed"] = opps_m2["amount"]
opps_m2.loc[opps_m2["amount_imputed"].isna(), "amount_imputed"] = (
    opps_m2.loc[opps_m2["amount_imputed"].isna(), "stage"].map(stage_medians)
)

# fallback to overall median if stage median missing
opps_m2["amount_imputed"] = opps_m2["amount_imputed"].fillna(overall_median)

# 3) Expected Value
# EV = amount * win_probability
opps_m2["ev"] = opps_m2["amount_imputed"] * opps_m2["win_probability_estimated"]

# 4) Executive defensibility snapshots
total_pipeline_amount = opps_m2["amount_imputed"].sum()
total_pipeline_ev = opps_m2["ev"].sum()

imputed_count = int(opps_m2["amount_imputed_flag"].sum())
imputed_pct = round(opps_m2["amount_imputed_flag"].mean() * 100, 2)

imputed_amount_share = round(
    (opps_m2.loc[opps_m2["amount_imputed_flag"], "amount_imputed"].sum() / total_pipeline_amount) * 100,
    2
)

print("EV model locked:")
print(f"- Opportunities: {len(opps_m2)}")
print(f"- Total Pipeline (Amount, €): {total_pipeline_amount:,.0f}")
print(f"- Total Pipeline EV (€, Amount × WinProb): {total_pipeline_ev:,.0f}")

print("\nMissing Amount Handling (CRM hygiene impact):")
print(f"- Missing amount (count): {imputed_count}")
print(f"- Missing amount (% of opps): {imputed_pct}%")
print(f"- Share of pipeline € from imputed values: {imputed_amount_share}%")

# Optional: stage-level imputation transparency
stage_impute_table = (
    opps_m2.groupby("stage")
    .agg(
        opp_count=("opp_id", "count"),
        missing_amount=("amount_imputed_flag", "sum"),
        median_amount_stage=("amount_imputed", "median"),
        pipeline_amount=("amount_imputed", "sum"),
        pipeline_ev=("ev", "sum")
    )
    .reset_index()
)

stage_impute_table["missing_amount_pct"] = (
    (stage_impute_table["missing_amount"] / stage_impute_table["opp_count"]) * 100
).round(2)

stage_impute_table = stage_impute_table.sort_values("pipeline_ev", ascending=False)

print("\nStage-level imputation + EV (top stages by EV):")
print(stage_impute_table.head(10))


EV model locked:
- Opportunities: 900
- Total Pipeline (Amount, €): 23,320,176
- Total Pipeline EV (€, Amount × WinProb): 10,707,050

Missing Amount Handling (CRM hygiene impact):
- Missing amount (count): 113
- Missing amount (% of opps): 12.56%
- Share of pipeline € from imputed values: 10.91%

Stage-level imputation + EV (top stages by EV):
         stage  opp_count  missing_amount  median_amount_stage  \
1   Closed Won        426              64              22369.5   
0  Closed Lost        257              24              22920.0   
3     Proposal        115              14              22933.0   
2  Negotiation        102              11              21856.0   

   pipeline_amount   pipeline_ev  missing_amount_pct  
1       11154028.0  5.836100e+06               15.02  
0        6460740.0  2.534350e+06                9.34  
3        3091420.0  1.357397e+06               12.17  
2        2613988.0  9.792032e+05               10.78  


In [11]:
print("Leads columns:")
print(list(leads.columns))

# Candidate duration columns commonly used in synthetic CRM datasets
duration_candidates = [
    "response_time_hours", "lead_response_hours", "response_hours",
    "time_to_first_response_hours", "first_response_hours",
    "first_response_time_hours", "sla_response_hours"
]

found_duration = [c for c in duration_candidates if c in leads.columns]

if found_duration:
    dur_col = found_duration[0]
    print(f"\nFound lead response duration column: {dur_col}")
    leads_sla = leads[["lead_id", dur_col]].copy()
    leads_sla[dur_col] = pd.to_numeric(leads_sla[dur_col], errors="coerce")
    leads_sla["slow_lead_response_flag"] = leads_sla[dur_col] > 48
    print("\nSLA flags computed from duration column.")
else:
    print("\nNo precomputed response-duration column found. Trying timestamp-based computation...")

    # Candidate timestamp pairs
    created_candidates = ["created_date", "lead_created_date", "lead_created_at", "created_at"]
    responded_candidates = ["first_response_date", "responded_date", "first_contacted_date",
                            "first_response_at", "responded_at", "first_contacted_at"]

    created_found = [c for c in created_candidates if c in leads.columns]
    responded_found = [c for c in responded_candidates if c in leads.columns]

    print("Created timestamp candidates found:", created_found)
    print("Responded timestamp candidates found:", responded_found)

    if created_found and responded_found:
        created_col = created_found[0]
        responded_col = responded_found[0]

        tmp = leads[["lead_id", created_col, responded_col]].copy()
        tmp[created_col] = pd.to_datetime(tmp[created_col], errors="coerce")
        tmp[responded_col] = pd.to_datetime(tmp[responded_col], errors="coerce")

        tmp["lead_response_hours_calc"] = (tmp[responded_col] - tmp[created_col]).dt.total_seconds() / 3600.0
        tmp["slow_lead_response_flag"] = tmp["lead_response_hours_calc"] > 48

        leads_sla = tmp[["lead_id", "lead_response_hours_calc", "slow_lead_response_flag"]].copy()

        print("\nComputed lead response hours from timestamps.")
        print(leads_sla["lead_response_hours_calc"].describe().round(2))
    else:
        print("\nBLOCKER: Leads table has neither a response-duration column nor usable timestamps.")
        print("We can still run leakage for other behaviours now, and add lead SLA leakage once schema supports it.")
        leads_sla = None

# Preview flags if available
if leads_sla is not None:
    print("\nSlow lead response rate (>48h):",
          round(leads_sla["slow_lead_response_flag"].mean() * 100, 2), "%")

Leads columns:
['lead_id', 'contact_id', 'account_id', 'lead_source', 'assigned_owner', 'created_date', 'mql_date', 'initial_response_time_hours', 'quality_true']

No precomputed response-duration column found. Trying timestamp-based computation...
Created timestamp candidates found: ['created_date']
Responded timestamp candidates found: []

BLOCKER: Leads table has neither a response-duration column nor usable timestamps.
We can still run leakage for other behaviours now, and add lead SLA leakage once schema supports it.


In [12]:
# ---- 1) Activity coverage flag ----
act_counts = acts.groupby("opp_id").size().rename("activity_count").reset_index()
opps_lkg = opps_m2.merge(act_counts, on="opp_id", how="left")
opps_lkg["activity_count"] = opps_lkg["activity_count"].fillna(0).astype(int)
opps_lkg["no_activity_flag"] = opps_lkg["activity_count"] == 0

# ---- 2) Stalled deal flag (>30 days) ----
opps_lkg["stalled_30d_flag"] = opps_lkg["days_in_stage_current"] > 30

# ---- 3) Stage movement flags ----
opps_lkg["stage_jump_flag"] = opps_lkg["stage_jump_flag"].fillna(False).astype(bool)
opps_lkg["backward_stage_flag"] = opps_lkg["backward_stage_change_count"].fillna(0) > 0

# ---- 4) Lead SLA join (if available) ----
if leads_sla is not None:
    opps_lkg = opps_lkg.merge(leads_sla[["lead_id", "slow_lead_response_flag"]], on="lead_id", how="left")
    opps_lkg["slow_lead_response_flag"] = opps_lkg["slow_lead_response_flag"].fillna(False).astype(bool)
else:
    opps_lkg["slow_lead_response_flag"] = False  # placeholder, not computed

# ---- 5) Haircut assumptions (explicit) ----
HAIRCUTS = {
    "slow_lead_response_flag": 0.15,
    "no_activity_flag": 0.20,
    "stage_jump_flag": 0.10,
    "stalled_30d_flag": 0.25
}

computed_flags = ["no_activity_flag", "stage_jump_flag", "stalled_30d_flag"]
if leads_sla is not None:
    computed_flags = ["slow_lead_response_flag"] + computed_flags

# ---- 6) Leakage per behaviour ----
rows = []
for flag in computed_flags:
    haircut = HAIRCUTS[flag]
    affected = opps_lkg[opps_lkg[flag]]
    rows.append({
        "behaviour": flag.replace("_flag", "").replace("_", " ").title(),
        "affected_opportunities": len(affected),
        "affected_pipeline_ev_eur": round(affected["ev"].sum(), 0),
        "assumed_ev_haircut_pct": int(haircut * 100),
        "estimated_ev_leakage_eur": round(affected["ev"].sum() * haircut, 0)
    })

leakage_table = pd.DataFrame(rows).sort_values("estimated_ev_leakage_eur", ascending=False)

leakage_table


,behaviour,affected_opportunities,affected_pipeline_ev_eur,assumed_ev_haircut_pct,estimated_ev_leakage_eur
2,Stalled 30D,678,7758752.0,25,1939688.0
0,No Activity,202,1946439.0,20,389288.0
1,Stage Jump,161,1931933.0,10,193193.0


In [13]:
# Build lead SLA flags from correct column
leads_sla = leads[["lead_id", "initial_response_time_hours"]].copy()
leads_sla["initial_response_time_hours"] = pd.to_numeric(leads_sla["initial_response_time_hours"], errors="coerce")

leads_sla["slow_lead_response_flag"] = leads_sla["initial_response_time_hours"] > 48
leads_sla["within_24h_flag"] = leads_sla["initial_response_time_hours"] <= 24
leads_sla["within_48h_flag"] = leads_sla["initial_response_time_hours"] <= 48

print("Lead SLA snapshot:")
print("Pct within 24h:", round(leads_sla["within_24h_flag"].mean() * 100, 2), "%")
print("Pct within 48h:", round(leads_sla["within_48h_flag"].mean() * 100, 2), "%")
print("Pct slow (>48h):", round(leads_sla["slow_lead_response_flag"].mean() * 100, 2), "%")

# Merge safely (handle existing placeholder column in opps_lkg)
opps_lkg2 = opps_lkg.merge(
    leads_sla[["lead_id", "slow_lead_response_flag"]],
    on="lead_id",
    how="left",
    suffixes=("_opps", "_leads")
)

# Detect which slow flag column exists after merge
slow_cols = [c for c in opps_lkg2.columns if "slow_lead_response_flag" in c]
print("\nSlow flag columns after merge:", slow_cols)

# Prefer the leads-derived one if present
if "slow_lead_response_flag_leads" in opps_lkg2.columns:
    opps_lkg2["slow_lead_response_flag"] = opps_lkg2["slow_lead_response_flag_leads"]
elif "slow_lead_response_flag" in opps_lkg2.columns:
    opps_lkg2["slow_lead_response_flag"] = opps_lkg2["slow_lead_response_flag"]
elif "slow_lead_response_flag_opps" in opps_lkg2.columns:
    opps_lkg2["slow_lead_response_flag"] = opps_lkg2["slow_lead_response_flag_opps"]
else:
    raise ValueError("No slow_lead_response_flag column found after merge — unexpected schema state.")

opps_lkg2["slow_lead_response_flag"] = opps_lkg2["slow_lead_response_flag"].fillna(False).astype(bool)

# Haircuts (explicit)
HAIRCUTS = {
    "slow_lead_response_flag": 0.15,
    "no_activity_flag": 0.20,
    "stage_jump_flag": 0.10,
    "stalled_30d_flag": 0.25
}

# Leakage table rebuild
rows = []
for flag, haircut in HAIRCUTS.items():
    affected = opps_lkg2[opps_lkg2[flag]]
    rows.append({
        "behaviour": flag.replace("_flag", "").replace("_", " ").title(),
        "affected_opportunities": len(affected),
        "affected_pipeline_ev_eur": round(affected["ev"].sum(), 0),
        "assumed_ev_haircut_pct": int(haircut * 100),
        "estimated_ev_leakage_eur": round(affected["ev"].sum() * haircut, 0)
    })

leakage_table_full = pd.DataFrame(rows).sort_values("estimated_ev_leakage_eur", ascending=False)

leakage_table_full

Lead SLA snapshot:
Pct within 24h: 55.0 %
Pct within 48h: 75.87 %
Pct slow (>48h): 24.13 %

Slow flag columns after merge: ['slow_lead_response_flag_opps', 'slow_lead_response_flag_leads']


,behaviour,affected_opportunities,affected_pipeline_ev_eur,assumed_ev_haircut_pct,estimated_ev_leakage_eur
3,Stalled 30D,678,7758752.0,25,1939688.0
1,No Activity,202,1946439.0,20,389288.0
0,Slow Lead Response,218,1689692.0,15,253454.0
2,Stage Jump,161,1931933.0,10,193193.0


In [ ]:

import pandas as pd
import numpy as np

FLAGS = ["slow_lead_response_flag", "no_activity_flag", "stage_jump_flag", "stalled_30d_flag"]

HAIRCUTS = {
    "slow_lead_response_flag": 0.15,
    "no_activity_flag": 0.20,
    "stage_jump_flag": 0.10,
    "stalled_30d_flag": 0.25
}

# 1) Deduped total leakage: apply max haircut per opp 
def max_haircut(row):
    hc = 0.0
    for f in FLAGS:
        if bool(row[f]):
            hc = max(hc, HAIRCUTS[f])
    return hc

opps_cf = opps_lkg2.copy()

opps_cf["max_haircut"] = opps_cf.apply(max_haircut, axis=1)
opps_cf["deduped_leakage_eur"] = opps_cf["ev"] * opps_cf["max_haircut"]

total_leakage_nondedup = leakage_table_full["estimated_ev_leakage_eur"].sum()
total_leakage_dedup = opps_cf["deduped_leakage_eur"].sum()

print("Leakage totals:")
print(f"- Non-deduped (sum of drivers): €{total_leakage_nondedup:,.0f}")
print(f"- Deduped (max haircut per opp): €{total_leakage_dedup:,.0f}")
print(f"- Deduped leakage as % of total EV (€10.71M): {round((total_leakage_dedup/10707050)*100, 2)}%")

# ---- 2) Scenario recovery rates (explicit) ----
RECOVERY_RATES = {
    "A_SLA_48H": {
        "slow_lead_response_flag": 0.60
    },
    "B_EVIDENCE_GATE": {
        "no_activity_flag": 0.50,
        "stage_jump_flag": 0.40
    },
    "C_STALL_GOVERNANCE": {
        "stalled_30d_flag": 0.40
    }
}

# ---- 3) Scenario recovery per opp (deduped within scenario) ----
def scenario_recovery(row, scenario_map):
    # compute potential recovery haircuts for triggered behaviours in scenario
    rec_hc = 0.0
    for f, rec_rate in scenario_map.items():
        if bool(row[f]):
            rec_hc = max(rec_hc, HAIRCUTS[f] * rec_rate)
    return row["ev"] * rec_hc

scenario_rows = []

for scen, fmap in RECOVERY_RATES.items():
    opps_cf[f"recovery_{scen}"] = opps_cf.apply(lambda r: scenario_recovery(r, fmap), axis=1)
    scenario_rows.append({
        "scenario": scen,
        "assumption_summary": ", ".join([f"{k.replace('_flag','').replace('_',' ')} @ {int(v*100)}% recovery"
                                         for k,v in fmap.items()]),
        "estimated_ev_leakage_recovered_eur": round(opps_cf[f"recovery_{scen}"].sum(), 0)
    })

scenario_table = pd.DataFrame(scenario_rows)

# ---- 4) Full package (A+B+C) — deduped across all included behaviours ----
full_map = {}
for scen in RECOVERY_RATES:
    full_map.update(RECOVERY_RATES[scen])

opps_cf["recovery_FULL_PACKAGE"] = opps_cf.apply(lambda r: scenario_recovery(r, full_map), axis=1)

full_row = pd.DataFrame([{
    "scenario": "D_FULL_PACKAGE",
    "assumption_summary": "Deduped per opp: max( haircut × recovery_rate ) across SLA + Evidence Gate + Stall Governance",
    "estimated_ev_leakage_recovered_eur": round(opps_cf["recovery_FULL_PACKAGE"].sum(), 0)
}])

scenario_table = pd.concat([scenario_table, full_row], ignore_index=True)

# ---- 5) Add context: recovered € as % of deduped leakage + total EV ----
scenario_table["recovered_pct_of_deduped_leakage"] = (
    scenario_table["estimated_ev_leakage_recovered_eur"] / total_leakage_dedup * 100
).round(2)

scenario_table["recovered_pct_of_total_ev"] = (
    scenario_table["estimated_ev_leakage_recovered_eur"] / opps_cf["ev"].sum() * 100
).round(2)

scenario_table = scenario_table.sort_values("estimated_ev_leakage_recovered_eur", ascending=False)

scenario_table


Leakage totals:
- Non-deduped (sum of drivers): €2,775,623
- Deduped (max haircut per opp): €2,084,151
- Deduped leakage as % of total EV (€10.71M): 19.47%


,scenario,assumption_summary,estimated_ev_leakage_recovered_eur,recovered_pct_of_deduped_leakage,recovered_pct_of_total_ev
3,D_FULL_PACKAGE,Deduped per opp: max( haircut × recovery_rate ...,844552.0,40.52,7.89
2,C_STALL_GOVERNANCE,stalled 30d @ 40% recovery,775875.0,37.23,7.25
1,B_EVIDENCE_GATE,"no activity @ 50% recovery, stage jump @ 40% r...",250428.0,12.02,2.34
0,A_SLA_48H,slow lead response @ 60% recovery,152072.0,7.30,1.42
